
# Gaussian Naive Bayes στο Iris dataset

Σε αυτό το notebook θα δούμε ένα απλό παράδειγμα χρήσης
**Gaussian Naive Bayes** στο κλασικό **Iris dataset**.

## Στόχοι

- Να θυμηθούμε τη βασική ιδέα του Naive Bayes.
- Να δούμε πότε χρησιμοποιούμε την παραλλαγή **Gaussian**.
- Να εκπαιδεύσουμε GaussianNB στο Iris dataset.
- Να αξιολογήσουμε την απόδοση με confusion matrix.
- Να δούμε posterior πιθανότητες για μερικά δείγματα.



## Θεωρία: Gaussian Naive Bayes

Υπενθυμίζουμε τον γενικό κανόνα του Naive Bayes:

$$
P(y \mid x) \propto P(y) P(x \mid y)
= P(y) \prod_i P(x_i \mid y).
$$

Στο **Gaussian Naive Bayes** θεωρούμε ότι κάθε χαρακτηριστικό $x_i$,
για κάθε κλάση $y = k$, ακολουθεί κανονική κατανομή:

$P(x_i \mid y = k) = \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2).$

Άρα η συνολική πιθανοφάνεια (likelihood) γράφεται:

$P(x \mid y = k) = \prod_i \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2)$

και επομένως η posterior (εκ των υστέρων):

$P(y = k \mid x) \propto P(y = k)
\prod_i \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2).$

Το Gaussian NB είναι κατάλληλο όταν τα χαρακτηριστικά είναι
**συνεχείς αριθμητικές μεταβλητές** (π.χ. μήκη, βάρη, φυσικές μετρήσεις κ.λπ.).



## Το Iris dataset

Το Iris dataset περιέχει μετρήσεις λουλουδιών:

- `sepal length`
- `sepal width`
- `petal length`
- `petal width`

και τρεις κλάσεις (είδη):

- `setosa`
- `versicolor`
- `virginica`

Είναι μικρό και "καθαρό" dataset, ιδανικό για
επίδειξη αλγορίθμων ταξινόμησης.


In [ ]:

# Βασικά imports

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, ConfusionMatrixDisplay


In [ ]:

# Φόρτωση του Iris dataset

iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
class_names = iris.target_names

print("Σχήμα X:", X.shape)
print("Χαρακτηριστικά:", feature_names)
print("Κλάσεις:", class_names)


In [ ]:

# Ρίχνουμε μια πρώτη ματιά στα δεδομένα (πρώτα 5 δείγματα)

X[:5], y[:5]


In [ ]:

# Απλή 2D απεικόνιση: petal length vs petal width

fig, ax = plt.subplots()
scatter = ax.scatter(X[:, 2], X[:, 3], c=y)
ax.set_xlabel("petal length")
ax.set_ylabel("petal width")
ax.set_title("Iris – petal length vs petal width")
fig.tight_layout()
plt.show()


In [ ]:

# Train / validation split και κανονικοποίηση με StandardScaler

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=0,
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Εκπαίδευση GaussianNB

gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train)

y_pred = gnb.predict(X_val_scaled)

print("=== Gaussian Naive Bayes στο Iris dataset ===")
print(
    classification_report(
        y_val,
        y_pred,
        target_names=class_names,
        digits=3,
    )
)


In [ ]:

# Confusion matrix

fig, ax = plt.subplots()
ConfusionMatrixDisplay.from_predictions(
    y_val,
    y_pred,
    display_labels=class_names,
    ax=ax,
    colorbar=False,
)
ax.set_title("Confusion matrix – Gaussian NB (Iris)")
fig.tight_layout()
plt.show()


In [ ]:

# Posterior πιθανότητες για μερικά δείγματα του validation set

idx_samples = [0, 5, 10]  # δείκτες μέσα στο X_val
X_samples = X_val[idx_samples]
X_samples_scaled = X_val_scaled[idx_samples]
y_true = y_val[idx_samples]

y_pred_samples = gnb.predict(X_samples_scaled)
y_proba_samples = gnb.predict_proba(X_samples_scaled)

for i, (x, true_label, pred_label, proba) in enumerate(
    zip(X_samples, y_true, y_pred_samples, y_proba_samples)
):
    print("-" * 72)
    print(f"Δείγμα #{i}")
    print(f"Χαρακτηριστικά (sepal_len, sepal_wid, petal_len, petal_wid): {x}")
    print(f"Πραγματική κλάση: {class_names[true_label]}")
    print(f"Πρόβλεψη       : {class_names[pred_label]}")
    print("Posterior πιθανότητες:")
    for cls_idx, cls_name in enumerate(class_names):
        print(f"  P({cls_name} | x) = {proba[cls_idx]:.3f}")
